### Import

In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import feature_selection 
from sklearn import preprocessing
from sklearn import metrics
from sklearn import neighbors
from sklearn import svm
from sklearn import decomposition
from sklearn.decomposition import PCA
from matplotlib import pyplot
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold


### Load data

In [3]:
def load_data():
    # this_directory = os.path.dirname(os.path.abspath(__file__))
    data = pd.read_csv(os.path.join('hn', 'HN_radiomicFeatures.csv'), index_col=0)

    label = data["label"]
    data = data.drop(["label"], axis=1)
    
    feature_names = data.columns.values
    return data, label, feature_names

data, label, feature_names = load_data()

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')


The number of samples: 113
The number of columns: 159


### Check for missing data

In [4]:
# Detect missing values in dataframe
data.isnull().values.any()

False

### Data split

In [5]:
def split_data(data, label):
    train_data, test_data, train_label, test_label = train_test_split(data, label, train_size=0.8)
    return train_data, test_data, train_label, test_label
# train_data, val_data, train_label, val_label = train_test_split(train_val_data, train_val_label, train_size=0.85)

train_data, test_data, train_label, test_label = split_data(data, label)
print(f'The number of train samples: {train_data.shape[0]}')
# print(f'The number of validation samples: {val_data.shape[0]}')
print(f'The number of test samples: {test_data.shape[0]}')

The number of train samples: 90
The number of test samples: 23


### Scaling train data

In [6]:
def scale_data(train_data, test_data, feature_names):
    scaler = StandardScaler().fit(train_data)
    train_data = scaler.transform(train_data)
    test_data = scaler.transform(test_data)
    train_data = pd.DataFrame(train_data, columns=feature_names)
    test_data = pd.DataFrame(test_data, columns=feature_names)
    return train_data, test_data

train_data, test_data = scale_data(train_data, test_data, feature_names)

### Lasso Regression

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel

def lasso_regression(train_data, train_label, test_data):
    sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
    sel_.fit(train_data, np.ravel(train_label,order='C'))
    sel_.get_support()
    train_data = pd.DataFrame(train_data)
    selected_feat = train_data.columns[(sel_.get_support())]
    lasso_train = train_data[selected_feat]
    lasso_test = test_data[selected_feat]
    return lasso_train, lasso_test

print('total features: {}'.format((train_data.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
np.sum(sel_.estimator_.coef_ == 0)))

removed_feats = train_data.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]

total features: 159


NameError: name 'selected_feat' is not defined

### Linear Classifier

In [ ]:
clf = SGDClassifier()
clf.fit(train_data, train_label)
print(np.shape(clf.coef_))
print(clf.intercept_)

label_train_pred = clf.predict(train_data)
print(metrics.accuracy_score(train_label, label_train_pred))

y_pred = clf.predict(test_data)
print(metrics.accuracy_score(test_label, y_pred))

(1, 157)
[-89.77098226]
1.0
0.6521739130434783


### kNN classifier

In [8]:
# Perform a PCA
def perform_pca(train_data, test_data, n_components):
    pca = decomposition.PCA(n_components=n_components)
    pca.fit(train_data)
    train_pca = pca.transform(train_data)
    test_pca = pca.transform(test_data)
    return pca, train_pca, test_pca

pca, train_pca, test_pca = perform_pca(train_data, test_data, 10)
print(sum(pca.explained_variance_ratio_))

# Fit kNN
def knn_classifier(train_pca, test_pca, train_label, test_label):
    knn = neighbors.KNeighborsClassifier(n_neighbors=10)
    knn.fit(train_pca, train_label)
    score_train = knn.score(train_pca, train_label)
    score_test = knn.score(test_pca, test_label)
    return score_train, score_test

score_train, score_test = knn_classifier(train_pca, test_pca, train_label, test_label)

# Print result
print(f"Training result: {score_train}")
print(f"Test result: {score_test}")

0.7150371834599125
Training result: 0.8111111111111111
Test result: 0.5217391304347826


In [9]:
def remove_zero_var(train_data, test_data, label):
    selector = VarianceThreshold()
    selector.fit_transform(train_data)
    zero_var_col = [column for column in train_data.columns if column not in train_data.columns[selector.get_support()]]
    train_data.drop(zero_var_col, inplace=True, axis=1)
    test_data.drop(zero_var_col, inplace=True, axis=1)
    label.drop(zero_var_col, inplace=True, axis=1)
    return train_data, test_data, label

In [10]:
# You can understand the F-Scores as a measure of how informative each feature is for your dataset.
# As it is explained in the method documentation, an F-test is carried out to assess each feature. 
# The F-scores are the test statistic for the F-test, and they basically represent the ratio between the explained and the unexplained variance.
kbest = 80

def select_features(kbest, train_data, train_label, test_data):
    fs = SelectKBest(score_func=f_classif, k=kbest)
    kbest_train = fs.fit_transform(train_data, train_label)
    kbest_test = fs.transform(test_data)
    return kbest_train, kbest_test

kbest_train, kbest_test = select_features(kbest, train_data, train_label, test_data)

C:\Users\velde\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [145 146] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
C:\Users\velde\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


### Find average accuracy over multiple classifications

In [11]:
# Find mean accuracy for kNN classification
loops = 100
score_train_array = np.zeros(loops)
score_test_array = np.zeros(loops)
num_best_features = 80
pca_components = 10

for i in range(loops):
    data, label, feature_names = load_data()
    train_data, test_data, train_label, test_label = split_data(data, label)
    train_data, test_data = scale_data(train_data, test_data, feature_names)
    train_data, test_data, label = remove_zero_var(train_data, test_data, label)

    # Find best features based on F-scores
    kbest_train, kbest_test = select_features(num_best_features, train_data, train_label, test_data)

    # Lasso regression
    # lasso_train, lasso_test = lasso_regression(kbest_train, train_label, kbest_test)
    
    # Perform a PCA
    pca, train_pca, test_pca = perform_pca(lasso_train, lasso_test, pca_components)

    # Fit kNN
    score_train, score_test = knn_classifier(train_pca, test_pca, train_label, test_label)
    score_train_array[i] = score_train
    score_test_array[i] = score_test


mean_score_train = np.mean(score_train_array)
mean_score_test = np.mean(score_test_array)
print(f"Mean training score: {mean_score_train}")
print(f"Mean test score: {mean_score_test}")


ValueError: No axis named 1 for object type Series

### Lasso Regression (L1)

      hf_min  hf_quartile_range  hf_skewness  of_theta_x  sf_convexity_avg_2D  \
0   0.029526           0.288815    -0.811220    1.162769            -1.742137   
1   0.045696          -0.281839    -1.209431   -0.923803            -0.545045   
2   0.464904          -0.697894    -0.334409   -0.591434            -2.275007   
3   0.998621           0.904870    -0.109128   -1.095027             1.118964   
4  -2.588238           1.019612    -0.108926    1.072188            -0.732455   
..       ...                ...          ...         ...                  ...   
85 -0.005356           0.516785    -0.916557    1.132639             0.760979   
86  1.123256          -0.820316    -1.809956   -0.010575            -0.250044   
87  0.818547           0.387321     2.417655    1.135878            -1.024006   
88 -0.027810          -0.186935    -0.952784    1.174060            -0.048489   
89  0.413851          -0.522116    -0.253373    1.094957             0.464221   

    sf_prax_avg_2D  sf_prax